# InternVideo 2 demo

It can be used to test the capabilities of InternVideo2 and to verify that the models are loaded correctly

In [ ]:
import pathlib
import sys
import os
sys.path.append(str(pathlib.Path(os.path.abspath('')).parent))

from tools.genrl_utils import viclip_global_instance
viclip_global_instance.instantiate()

In [ ]:
import cv2
import numpy as np
import torch
from tools.genrl_utils import INTERNVIDEO_PATH

def _frame_from_video(video):
    while video.isOpened():
        success, frame = video.read()
        if success:
            yield frame
        else:
            break

ASSET_PATH = pathlib.Path(os.path.abspath('')).parent / 'assets'

# 83 % -  A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.
video = cv2.VideoCapture( str(INTERNVIDEO_PATH / 'InternVideo2/multi_modality/demo/example1.mp4') )
# # 99 % -  A karate kick
# video = cv2.VideoCapture( str( ASSET_PATH / 'video_samples/karate_kick.mp4') ) 
# # 99 % -  A headstand
# video = cv2.VideoCapture( str( ASSET_PATH / 'video_samples/headstand.mp4') ) 

frames = [x for x in _frame_from_video(video)]
processed_frames = viclip_global_instance.viclip.preprocess_transf(torch.from_numpy(np.stack(frames[:8], axis=0)).permute(0,3,1,2) / 255)
frames_tensor = processed_frames.reshape(1, 8, 3, 224,224)

In [ ]:
text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery.",
                   "A person playing with a kid in the street",
                   "A group of friends playing bowling.",
                   "A japanese girl eating noodles",
                   "A painting by Monet",
                   "A karate kick",
                   "A headstand"]

In [ ]:
text_feat = viclip_global_instance.viclip.get_txt_feat(text_candidates)
video_feat = viclip_global_instance.viclip.get_vid_features(frames_tensor.to(viclip_global_instance.viclip.device))

sorted_probs, sorted_idxs = (100.0 * video_feat @ text_feat.T).softmax(dim=-1)[0].topk(len(text_feat))

for p, i in zip(sorted_probs, sorted_idxs):
    if p > 0.01:
        print(int(p * 100), '% - ', text_candidates[i])